# Query -> Hallucinated Classification -> Real Classification

**NOTE** this is here for reference. To actually run the notebook, [run it in google colab](https://colab.research.google.com/drive/1-xX7eXiDfG6_BDOIGIMQ5Ch3ws80kJsC)

<small>
(from <a href="http://maven.com/softwaredoug/cheat-at-search">Cheat at Search with LLMs</a> training course by Doug Turnbull.)
</small>

We previously constrained the LLM to a set of outputs using `Literal` what if we just ask the LLM to return a fake classification that looks like one of ours. Then we use an embedding to resolve to the "real" entity.

We can do this when we have a large-ish set of labels (brands, etc) to do a kind of large multi-label classification

**Refinement** -- Let the model hallucinate a classification, then do an embedding lookup to get a real classification

## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive)

In [ ]:
import os
os.environ["CHEAT_AT_SEARCH_DATA_PATH"] = "/home/jovyan/data"

from cheat_at_search.data_dir import mount
mount(use_gdrive=True)
from cheat_at_search.search import run_strategy, graded_bm25, ndcgs, ndcg_delta, vs_ideal
from cheat_at_search.wands_data import products

CAT_SEP = "/"

products

## Query -> Full classification

Our usual list of classifications

**EXCEPT** - we're not passing them to the LLM, we're building an embedding index of them on this line

```
model = SentenceTransformer('all-MiniLM-L6-v2')
real_classifications = model.encode(classifications_list)
top_level_category_list = np.asarray([c.split(" / ")[0].strip() for c in classifications_list])
```

**Lookup** we also do a lookup in `classifications` to map a hallucinated classification to our real classifications

```
    @property
    def classifications(self):
        """Search with model to resolve to real classification."""
        actuals = []
        for halluc_class in self.hallucinated_classification:
            top_level_halluc_category = halluc_class.split(" / ")[0].strip()

            query_embedding = model.encode(halluc_class)
            dot_prods = np.dot(real_classifications, query_embedding)
            actual = classifications_list[np.argmax(dot_prods)]
            while actual in actuals:
                dot_prods[np.argmax(dot_prods)] = -1
                actual = classifications_list[np.argmax(dot_prods)]
```

In [ ]:
import numpy as np

classifications_list = np.asarray([
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs',
 'Rugs / Area Rugs',
 'Furniture / Office Furniture / Desks',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / End & Side Tables',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows',
 'Furniture / Bedroom Furniture / Dressers & Chests',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Conversation Sets',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities',
 'Furniture / Living Room Furniture / Console Tables',
 'Décor & Pillows / Art / All Wall Art',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs',
 'Furniture / Office Furniture / Office Chairs',
 'Décor & Pillows / Mirrors / All Mirrors',
 'Bed & Bath / Bedding / All Bedding',
 'Décor & Pillows / Wall Décor / Wall Accents',
 'Furniture / Living Room Furniture / Chairs & Seating / Recliners',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen and Dining Sets',
 'Décor & Pillows / Window Treatments / Curtains & Drapes',
 'Furniture / Living Room Furniture / Sectionals',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Kids Beds',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / TV Stands & Entertainment Centers',
 'Lighting / Ceiling Lights / Chandeliers',
 'Furniture / Bedroom Furniture / Nightstands',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Kids Desks',
 'Décor & Pillows / Home Accessories / Decorative Objects',
 'Furniture / Bedroom Furniture / Beds & Headboards / Headboards',
 'Furniture / Living Room Furniture / Sofas',
 'Furniture / Living Room Furniture / Cabinets & Chests',
 'Décor & Pillows / Clocks / Wall Clocks',
 'Storage & Organization / Bathroom Storage & Organization / Bathroom Cabinets & Shelving',
 'Lighting / Table & Floor Lamps / Table Lamps',
 'Furniture / Living Room Furniture / Ottomans & Poufs',
 'Furniture / Kitchen & Dining Furniture / Kitchen Islands & Carts',
 'Furniture / Living Room Furniture / Bookcases',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Sofas & Sectionals',
 'Furniture / Office Furniture / Office Storage Cabinets',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Tables',
 'Contractor / Entry & Hallway / Coat Racks & Umbrella Stands',
 'Bed & Bath / Bedding Essentials / Mattress Pads & Toppers',
 'Home Improvement / Hardware / Home Hardware / Switch Plates',
 'Baby & Kids / Toddler & Kids Playroom / Playroom Furniture / Toddler & Kids Chairs & Seating',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Outdoor Covers / Patio Furniture Covers',
 'Rugs / Doormats',
 'Rugs / Kitchen Mats',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Queen Size Beds',
 'Furniture / Bedroom Furniture / Daybeds',
 'Furniture / Living Room Furniture / Living Room Sets',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Dining Sets',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Single Hole Bathroom Sink Faucets',
 'Outdoor / Outdoor Décor / Statues & Sculptures',
 'Décor & Pillows / Art / All Wall Art / Green Wall Art',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Table Sets',
 'Furniture / Living Room Furniture / Chairs & Seating / Chaise Lounge Chairs',
 'Storage & Organization / Wall Shelving & Organization / Wall and Display Shelves',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Rectangle Coffee Tables',
 'Décor & Pillows / Art / All Wall Art / Brown Wall Art',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools / Counter (24-27) Bar Stools & Counter Stools',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Plant Stands & Tables',
 'Décor & Pillows / Window Treatments / Curtain Hardware & Accessories',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Chairs / Side Kitchen & Dining Chairs',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Outdoor Club Chairs',
 'Furniture / Living Room Furniture / Chairs & Seating / Benches',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Sinks / Farmhouse & Apron Kitchen Sinks',
 'Kitchen & Tabletop / Kitchen Organization / Food Pantries',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel & Robe Hooks / Black Towel & Robe Hooks',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Deck Boxes & Patio Storage',
 'Outdoor / Garden / Planters',
 'Lighting / Wall Lights / Bathroom Vanity Lighting',
 'Furniture / Kitchen & Dining Furniture / Sideboards & Buffets',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Storage Racks & Shelving Units',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Bronze Cabinet & Drawer Pulls',
 'Storage & Organization / Storage Containers & Drawers / All Storage Containers',
 'Bed & Bath / Shower Curtains & Accessories / Shower Curtains & Shower Liners',
 'Storage & Organization / Bathroom Storage & Organization / Hampers & Laundry Baskets',
 'Lighting / Light Bulbs & Hardware / Light Bulbs / All Light Bulbs / LED Light Bulbs',
 'Décor & Pillows / Art / All Wall Art / Blue Wall Art',
 'Bed & Bath / Mattresses & Foundations / Innerspring Mattresses',
 'Lighting / Outdoor Lighting / Outdoor Wall Lighting',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Natural Material Storage / Log Storage',
 'Bed & Bath / Bathroom Accessories & Organization / Countertop Bath Accessories',
 'Storage & Organization / Shoe Storage / All Shoe Storage',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Ceramic Floor Tiles & Wall Tiles',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Black Cabinet & Drawer Pulls',
 'Bed & Bath / Mattresses & Foundations / Adjustable Beds',
 "Rugs / Area Rugs / 2' x 3' Area Rugs",
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Twin Beds',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Widespread Bathroom Sink Faucets',
 "Rugs / Area Rugs / 4' x 6' Area Rugs",
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets',
 'Kitchen & Tabletop / Tableware & Drinkware / Table & Kitchen Linens / All Table Linens',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Food Storage Containers',
 'Décor & Pillows / Flowers & Plants / Faux Flowers',
 'Bed & Bath / Bedding / All Bedding / Twin Bedding',
 'Furniture / Bedroom Furniture / Dressers & Chests / White Dressers & Chests',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Porcelain Floor Tiles & Wall Tiles',
 'Home Improvement / Flooring, Walls & Ceiling / Flooring Installation & Accessories / Molding & Millwork / Wall Molding & Millwork',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Barn Door Hardware',
 'Bed & Bath / Bedding / Sheets & Pillowcases',
 'Furniture / Office Furniture / Chair Mats / Hard Floor Chair Mats',
 'Outdoor / Outdoor Fencing & Flooring / All Fencing',
 'Storage & Organization / Closet Storage & Organization / Clothes Racks & Garment Racks',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Colanders, Strainers, & Salad Spinners',
 'Outdoor / Hot Tubs & Saunas / Saunas',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Blue Throw Pillows',
 'Bed & Bath / Bedding Essentials / Bed Pillows',
 'Lighting / Wall Lights / Wall Sconces',
 'Outdoor / Front Door Décor & Curb Appeal / Mailboxes',
 'Outdoor / Garden / Greenhouses',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Faucets & Systems',
 'Bed & Bath / Mattresses & Foundations / Queen Mattresses',
 'Furniture / Bedroom Furniture / Jewelry Armoires',
 'Outdoor / Outdoor Shades / Awnings',
 'Baby & Kids / Nursery Bedding / Crib Bedding Sets',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Knobs / Brass Cabinet & Drawer Knobs',
 'Décor & Pillows / Art / All Wall Art / Red Wall Art',
 'Lighting / Ceiling Lights / All Ceiling Lights',
 'Lighting / Light Bulbs & Hardware / Lighting Components',
 'Furniture / Game Tables & Game Room Furniture / Poker & Card Tables',
 'Appliances / Kitchen Appliances / Range Hoods / All Range Hoods',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Natural Stone Floor Tiles & Wall Tiles',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Bar Stools & Counter Stools / All Bar Stools & Counter Stools / Bar (28-33) Bar Stools & Counter Stools',
 'Outdoor / Outdoor Cooking & Tableware / Outdoor Serving & Tableware / Coolers, Baskets & Tubs / Picnic Baskets & Backpacks',
 'Décor & Pillows / Picture Frames & Albums / All Picture Frames',
 'Bed & Bath / Shower Curtains & Accessories / Shower Curtain Hooks',
 'Outdoor / Outdoor Shades / Outdoor Umbrellas / Patio Umbrella Stands & Bases',
 'Outdoor / Outdoor & Patio Furniture / Patio Bar Furniture / Patio Bar Stools',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Toilets & Bidets / Toilet Paper Holders / Free Standing Toilet Paper Holders',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Bike & Sport Racks',
 'Appliances / Kitchen Appliances / Refrigerators & Freezers / All Refrigerators / French Door Refrigerators',
 'Décor & Pillows / Home Accessories / Decorative Trays',
 'School Furniture and Supplies / School Spaces / Computer Lab Furniture / Podiums & Lecterns',
 'Lighting / Light Bulbs & Hardware / Lighting Shades',
 'Furniture / Kitchen & Dining Furniture / Bar Furniture / Home Bars & Bar Sets',
 'Lighting / Table & Floor Lamps / Floor Lamps',
 'Décor & Pillows / Wall Décor / Wall Accents / Brown Wall Accents',
 'Kitchen & Tabletop / Small Kitchen Appliances / Pressure & Slow Cookers / Slow Cookers / Slow Slow Cookers',
 'Décor & Pillows / Window Treatments / Curtains & Drapes / 90 Inch Curtains & Drapes',
 'Furniture / Bedroom Furniture / Armoires & Wardrobes',
 'Kitchen & Tabletop / Tableware & Drinkware / Flatware & Cutlery / Serving Utensils',
 'Baby & Kids / Baby & Kids Décor & Lighting / All Baby & Kids Wall Art',
 'Furniture / Office Furniture / Desks / Writing Desks',
 'Furniture / Office Furniture / Office Chairs / Task Office Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Shower & Bathtub Doors',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Rocking Chairs & Gliders',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Wall Paneling',
 'Outdoor / Garden / Plant Stands & Accessories',
 'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Tables / 4 Seat Kitchen & Dining Tables',
 'Décor & Pillows / Home Accessories / Vases, Urns, Jars & Bottles',
 'Lighting / Wall Lights / Under Cabinet Lighting / Strip Under Cabinet Lighting',
 'Furniture / Bedroom Furniture / Bedroom and Makeup Vanities',
 'Pet / Dog / Dog Bowls & Feeding Supplies / Pet Bowls & Feeders',
 'Décor & Pillows / Candles & Holders / Candle Holders',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Shower & Bathtub Accessories',
 'Furniture / Office Furniture / Office Chair Accessories / Seat Cushion Office Chair Accessories',
 'Furniture / Office Furniture / Chair Mats',
 'Furniture / Living Room Furniture / Chairs & Seating / Massage Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities / Modern & Contemporary Bathroom Vanities',
 'Lighting / Ceiling Fans / All Ceiling Fans',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Black Kitchen Faucets',
 'Lighting / Light Bulbs & Hardware / Light Bulbs / All Light Bulbs / Incandescent Light Bulbs',
 'Home Improvement / Flooring, Walls & Ceiling / Flooring Installation & Accessories / Molding & Millwork',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Bathtubs',
 'Décor & Pillows / Art / All Wall Art / Yellow Wall Art',
 'Pet / Dog / Pet Gates, Fences & Doors / Pet Gates',
 'Furniture / Bedroom Furniture / Beds & Headboards / Bed Frames / Twin Bed Frames',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel Bars, Racks, and Stands / Metal Towel Bars, Racks, and Stands',
 'Décor & Pillows / Art / All Wall Art / Pink Wall Art',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Smoke Detectors / Wall & Ceiling Mounted Smoke Detectors',
 'Outdoor / Garden / Planters / Plastic Planters',
 'Décor & Pillows / Mirrors / All Mirrors / Accent Mirrors',
 'Appliances / Kitchen Appliances / Range Hoods / All Range Hoods / Wall Mount Range Hoods',
 'Outdoor / Garden / Garden Décor / Lawn & Garden Accents',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Round Coffee Tables',
 'Kitchen & Tabletop / Tableware & Drinkware / Dinnerware / Dining Bowls',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads / Dual Shower Heads',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Glass Floor Tiles & Wall Tiles',
 'School Furniture and Supplies / Facilities & Maintenance / Trash & Recycling',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Nickel Cabinet & Drawer Pulls',
 'Storage & Organization / Closet Storage & Organization / Closet Systems',
 'Furniture / Bedroom Furniture / Beds & Headboards / Beds / Full & Double Beds',
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands / Printer Carts & Stands',
 'Storage & Organization / Closet Storage & Organization / Closet Accessories',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities / Traditional Bathroom Vanities',
 'Home Improvement / Plumbing / Core Plumbing / Parts & Components',
 'Holiday Décor / Christmas / Christmas Trees / All Christmas Trees',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Black Throw Pillows',
 'Furniture / Game Tables & Game Room Furniture / Sports Team Fan Shop & Memorabillia / Life Size Cutouts',
 'Lighting / Ceiling Lights / Pendant Lighting',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Towel Storage / Towel & Robe Hooks',
 'Appliances / Washers & Dryers / Dryers / All Dryers / Gas Dryers',
 'Outdoor / Outdoor Recreation / Backyard Play / Kids Cars & Ride-On Toys',
 'Kitchen & Tabletop / Small Kitchen Appliances / Coffee, Espresso, & Tea / Coffee Makers',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Patio Sofas & Sectionals / Sectional Patio Sofas & Sectionals',
 'Lighting / Wall Lights / Under Cabinet Lighting',
 'Foodservice / Foodservice Tables / Table Parts',
 'Lighting / Outdoor Lighting / Landscape Lighting / All Landscape Lighting / Fence Post Cap Landscape Lighting',
 'Lighting / Outdoor Lighting / Landscape Lighting / All Landscape Lighting',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Tables / All Patio Tables',
 'Commercial Business Furniture / Commercial Office Furniture / Office Storage & Filing / Office Carts & Stands / All Carts & Stands / Utility Carts & Stands',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Outdoor Chaise & Lounge Chairs',
 'Furniture / Living Room Furniture / Chairs & Seating / Recliners / Brown Recliners',
 'Pet / Bird / Bird Perches & Play Gyms',
 'Décor & Pillows / Picture Frames & Albums / All Picture Frames / Single Picture Picture Frames',
 'Lighting / Outdoor Lighting / Outdoor Lanterns & Lamps',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls',
 'Bed Accessories',
 'Clips/Clamps',
 'Décor & Pillows / Wall Décor / Wall Decals',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles',
 'Bed & Bath / Bedding / Sheets & Pillowcases / Twin XL Sheets & Pillowcases',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Serving Trays & Boards / Serving Trays & Platters / Serving Serving Trays & Platters',
 'Holiday Décor / Holiday Lighting',
 'Décor & Pillows / Wall Décor / Memo Boards',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Toilets & Bidets / Toilet Paper Holders / Wall Mounted Toilet Paper Holders',
 'Décor & Pillows / Window Treatments / Curtains & Drapes / 63 Inch and Less Curtains & Drapes',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Knobs / Egg Door Knobs',
 'Décor & Pillows / Clocks / Wall Clocks / Analog Wall Clocks',
 'Home Improvement / Doors & Door Hardware / Interior Doors / Sliding Interior Doors',
 'Outdoor / Outdoor Recreation / Outdoor Games / All Outdoor Games',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Levers / Round Door Levers',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Sheds / Storage Sheds',
 'Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Door Levers',
 'School Furniture and Supplies / School Furniture / School Tables / Folding Tables / Wood Folding Tables',
 'Décor & Pillows / Wall Décor / Wall Accents / Green Wall Accents',
 'School Furniture and Supplies / Facilities & Maintenance / Commercial Signage',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Garage Storage Cabinets',
 'Furniture / Bedroom Furniture / Dressers & Chests / Beige Dressers & Chests',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves',
 'Furniture / Game Tables & Game Room Furniture / Dartboards & Cabinets',
 'Outdoor / Outdoor Décor / Outdoor Pillows & Cushions / Patio Furniture Cushions / Lounge Chair Patio Furniture Cushions',
 'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Dining Sets / Two Person Patio Dining Sets',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / Ivory & Cream Throw Pillows',
 'Appliances / Washers & Dryers / Washer & Dryer Sets / Black Washer & Dryer Sets',
 'School Furniture and Supplies / School Furniture / School Chairs & Seating / Stackable Chairs',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Pulls / Brass Cabinet & Drawer Pulls',
 'School Furniture and Supplies / School Boards & Technology / AV, Mounts & Tech Accessories / Electronic Mounts & Stands / Computer Mounts',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Papasan Accent Chairs',
 'Storage & Organization / Shoe Storage / All Shoe Storage / Rack Shoe Storage',
 'Storage & Organization / Shoe Storage / All Shoe Storage / Cabinet Shoe Storage',
 'Storage & Organization / Storage Containers & Drawers / Storage Drawers',
 'Appliances / Kitchen Appliances / Wine & Beverage Coolers / Water Coolers',
 'Furniture / Living Room Furniture / Chairs & Seating / Rocking Chairs',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Serving Bowls & Baskets / Serving Bowls / NA Serving Bowls',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / Projection Screens / Inflatable Projection Screens',
 'Appliances / Kitchen Appliances / Large Appliance Parts & Accessories',
 'Storage & Organization / Bathroom Storage & Organization / Hampers & Laundry Baskets / Laundry Hampers & Laundry Baskets',
 'Furniture / Office Furniture / Office Stools',
 'Outdoor / Outdoor & Patio Furniture / Outdoor Seating & Patio Chairs / Patio Seating / Outdoor Club Chairs / Metal Outdoor Club Chairs',
 'School Furniture and Supplies / School Furniture / School Tables / Folding Tables',
 'Lighting / Wall Lights / Bathroom Vanity Lighting / Traditional Bathroom Vanity Lighting',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Sinks & Faucet Components / Bathroom Sink Faucets / Centerset Bathroom Sink Faucets',
 'Décor & Pillows / Flowers & Plants / Faux Flowers / Orchid Faux Flowers',
 'Home Improvement / Flooring, Walls & Ceiling / Floor Tiles & Wall Tiles / Metal Floor Tiles & Wall Tiles',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Sinks',
 'Storage & Organization / Garage & Outdoor Storage & Organization / Outdoor Covers / Grill Covers / Charcoal Grill Grill Covers',
 'Outdoor / Outdoor Décor / Outdoor Wall Décor',
 'Storage & Organization / Cleaning & Laundry Organization / Laundry Room Organizers',
 'Reception Area / Reception Seating / Reception Sofas & Loveseats',
 'Kitchen & Tabletop / Cookware & Bakeware / Baking Sheets & Pans / Bread & Loaf Pans / Steel Bread & Loaf Pans',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Wingback Accent Chairs',
 'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Showers & Bathtubs / Showers & Bathtubs Plumbing / Shower Heads / Fixed Shower Heads',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Kitchen Gadgets / Pasta Makers & Accessories',
 'School Furniture and Supplies / School Furniture / School Chairs & Seating / Classroom Chairs / High School & College Classroom Chairs',
 'Furniture / Living Room Furniture / Sectionals / Stationary Sectionals',
 'Furniture / Kitchen & Dining Furniture / Sideboards & Buffets / Drawer Equipped Sideboards & Buffets',
 'Kitchen & Tabletop / Cookware & Bakeware / Baking Sheets & Pans / Bread & Loaf Pans',
 'Kitchen & Tabletop / Kitchen Utensils & Tools / Cooking Utensils / All Cooking Utensils / Kitchen Cooking Utensils',
 'Décor & Pillows / Flowers & Plants / Live Plants',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / Projection Screens / Folding Frame Projection Screens',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Metal Kitchen Canisters & Jars',
 'Outdoor / Outdoor Décor / Outdoor Fountains',
 'Outdoor / Outdoor Shades / Pergolas / Wood Pergolas',
 'Décor & Pillows / Candles & Holders / Candle Holders / Sconce Candle Holders',
 'Kitchen & Tabletop / Tableware & Drinkware / Serveware / Cake & Tiered Stands',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Chrome Kitchen Faucets',
 'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows / White Throw Pillows',
 'Outdoor / Outdoor Fencing & Flooring / Turf',
 'Décor & Pillows / Window Treatments / Valances & Kitchen Curtains',
 'Home Improvement / Hardware / Cabinet Hardware / Cabinet & Drawer Knobs / Black Cabinet & Drawer Knobs',
 'Home Improvement / Kitchen Remodel & Kitchen Fixtures / Kitchen Sinks & Faucet Components / Kitchen Faucets / Bronze Kitchen Faucets',
 'Appliances / Washers & Dryers / Washer & Dryer Sets',
 'Décor & Pillows / Clocks / Mantel & Tabletop Clocks',
 'Home Improvement / Doors & Door Hardware / Interior Doors',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves / Floating Wall & Display Shelves',
 'Outdoor / Outdoor Recreation / Backyard Play / Climbing Toys & Slides',
 'Home Improvement / Building Equipment / Dollies / Hand Truck Dollies',
 'Baby & Kids / Toddler & Kids Bedroom Furniture / Baby & Kids Dressers',
 'Décor & Pillows / Mirrors / All Mirrors / Leaning & Floor Mirrors',
 'Kitchen & Tabletop / Tableware & Drinkware / Drinkware / Mugs & Teacups',
 'Décor & Pillows / Flowers & Plants / Wreaths',
 'Outdoor / Outdoor Shades / Pergolas / Metal Pergolas',
 'Bed & Bath / Bedding / Sheets & Pillowcases / Twin Sheets & Pillowcases',
 'Outdoor / Outdoor Shades / Pergolas',
 'Reception Area / Reception Seating / Office Sofas & Loveseats',
 'Décor & Pillows / Home Accessories / Indoor Fountains',
 'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters / Kitchen Canisters & Jars / Ceramic Kitchen Canisters & Jars',
 'Décor & Pillows / Window Treatments / Curtain Hardware & Accessories / Bracket Curtain Hardware & Accessories',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Accent Tiles / Ceramic Accent Tiles',
 'Home Improvement / Flooring, Walls & Ceiling / Walls & Ceilings / Accent Tiles',
 'Furniture / Living Room Furniture / Chairs & Seating / Accent Chairs / Arm Accent Chairs',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Free Form Coffee Tables',
 'Décor & Pillows / Flowers & Plants / Faux Flowers / Rose Faux Flowers',
 'Bed & Bath / Mattresses & Foundations / Innerspring Mattresses / Twin Innerspring Mattresses',
 'Outdoor / Outdoor Décor / Outdoor Pillows & Cushions / Patio Furniture Cushions / Dining Chair Patio Furniture Cushions',
 'Furniture / Living Room Furniture / TV Stands & Media Storage Furniture / TV Stands & Entertainment Centers / Traditional TV Stands & Entertainment Centers',
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Plant Stands & Tables / Square Plant Stands & Tables',
 'Storage & Organization / Wall Shelving & Organization / Wall & Display Shelves / Corner Wall & Display Shelves',
 "Rugs / Area Rugs / 3' x 5' Area Rugs",
 'Kitchen & Tabletop / Tableware & Drinkware / Drinkware / Mugs & Teacups / Coffee Mugs & Teacups',
 'Contractor / Entry & Hallway / Coat Racks & Umbrella Stands / Wall Mounted Coat Racks & Umbrella Stands',
 "Baby & Kids / Toddler & Kids Playroom / Indoor Play / Kids' Playhouses",
 'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables / Square Coffee Tables',
 'Baby & Kids / Toddler & Kids Playroom / Indoor Play / Dollhouses & Accessories',
 'Bed & Bath / Bedding / All Bedding / Queen Bedding',
])

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
real_classifications = model.encode(classifications_list)
top_level_category_list = np.asarray([c.split(CAT_SEP)[0].strip() for c in classifications_list])
top_level_category_list

array(['Furniture', 'Furniture', 'Rugs', 'Furniture', 'Furniture',
       'Furniture', 'Décor & Pillows', 'Furniture', 'Outdoor',
       'Home Improvement', 'Furniture', 'Décor & Pillows', 'Furniture',
       'Furniture', 'Furniture', 'Décor & Pillows', 'Bed & Bath',
       'Décor & Pillows', 'Furniture', 'Furniture', 'Décor & Pillows',
       'Furniture', 'Baby & Kids', 'Furniture', 'Lighting', 'Furniture',
       'Baby & Kids', 'Décor & Pillows', 'Furniture', 'Furniture',
       'Furniture', 'Décor & Pillows', 'Storage & Organization',
       'Lighting', 'Furniture', 'Furniture', 'Furniture', 'Outdoor',
       'Furniture', 'Furniture', 'Contractor', 'Bed & Bath',
       'Home Improvement', 'Baby & Kids', 'Storage & Organization',
       'Rugs', 'Rugs', 'Furniture', 'Furniture', 'Furniture', 'Outdoor',
       'Home Improvement', 'Outdoor', 'Décor & Pillows', 'Furniture',
       'Furniture', 'Storage & Organization', 'Furniture',
       'Décor & Pillows', 'Furniture', 'Furniture', 'Déc

### Structured types

* Returns a `hallucinated_classification` that we turn into a real classification via finding the most similar embedding

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
from cheat_at_search.enrich import AutoEnricher
import numpy as np

known_categories = set([c.split(CAT_SEP)[0].strip() for c in classifications_list])
known_sub_categories = set([c.split(CAT_SEP)[1].strip() for c in classifications_list if len(c.split(CAT_SEP)) > 1])

known_sub_categories


class Query(BaseModel):
    """
    Base model for search queries, containing common query attributes.
    """
    keywords: str = Field(
        ...,
        description="The original search query keywords sent in as input"
    )


CAT_SEP = '/'


class QueryClassification(Query):
    """
    Represents a classification of a product.

    In this case, hallucinated, something the model is making up that looks like one
    of our classifications
    """
    hallucinated_classification: List[str] = Field(
        description="The classification you created for the query."
    )

    @property
    def classifications(self):
        """Search with model to resolve to real classification."""
        actuals = []
        for halluc_class in self.hallucinated_classification:
            query_embedding = model.encode(halluc_class)
            dot_prods = np.dot(real_classifications, query_embedding)
            actual = classifications_list[np.argmax(dot_prods)]
            while actual in actuals:
                dot_prods[np.argmax(dot_prods)] = -1
                actual = classifications_list[np.argmax(dot_prods)]

            actuals.append(actual)
        return actuals

    @property
    def categories(self):
        resolved_classification = self.classifications
        if len(resolved_classification) == 0 or resolved_classification == ["No Classification Fits"]:
            return []
        cats = []
        for c in resolved_classification:
            cats.append(c.split(CAT_SEP)[0].strip())
        return set(cats)

    @property
    def sub_categories(self):
        resolved_classification = self.classifications
        if len(resolved_classification) == 0 or resolved_classification == ["No Classification Fits"]:
            return []

        cats = []
        for c in resolved_classification:
            c_split = c.split(CAT_SEP)
            if len(c_split) < 2:
                continue
            cats.append(c.split(CAT_SEP)[1].strip())
        return set(cats)

    @property
    def cat_subcat(self):
        resolved_classification = self.classifications
        if len(resolved_classification) == 0 or resolved_classification == ["No Classification Fits"]:
            return []
        cat_subcats = []
        for c in resolved_classification:
            c_split = c.split(CAT_SEP)
            if len(c_split) < 2:
                continue
            cat_subcats.append(f"{c_split[0].strip()} / {c_split[1].strip()}")
        return set(cat_subcats)


### Query classification code

Notice our prompt:
1. Is given some examples of what classifications might look like
2. Asked to "be creative" to give us a diverse set of top level categories

In [ ]:
enricher = AutoEnricher(
     model="openai/gpt-4.1-nano",
     system_prompt="Your task is to create novel, never seen before, furniture, home goods, or hardware classification that best fit a search query. ",
     response_model=QueryClassification
)

def get_prompt_fully_qualified(query):
        prompt = f"""

        Some inspiration on what these look like is at the bottom.

        Here is the users request:

        {query}

        Product classifications might look like:

        'Furniture / Living Room Furniture / Coffee Tables & End Tables / Coffee Tables'
        'Décor & Pillows / Decorative Pillows & Blankets / Throw Pillows'
        'Furniture / Bedroom Furniture / Dressers & Chests'
        'Outdoor / Outdoor & Patio Furniture / Patio Furniture Sets / Patio Conversation Sets'
        'Home Improvement / Bathroom Remodel & Bathroom Fixtures / Bathroom Vanities / All Bathroom Vanities'
        'Lighting / Wall Lights / Bathroom Vanity Lighting'
        'Kitchen & Tabletop / Kitchen Organization / Food Storage & Canisters'
        'School Furniture and Supplies / School Furniture / School Chairs & Seating / Stackable Chairs',
        'Baby & Kids / Toddler & Kids Bedroom Furniture / Kids Beds',

        If you feel inspired, return many unique values in a list. Be creative. Cast a wide net with related but diverse categories.

        Return empty list if no clear classification could be inferred, the query is not clearly a furniture query

        """

        return prompt

def fully_classified(query):
    prompt = get_prompt_fully_qualified(query)
    return enricher.enrich(prompt)


def get_num_tokens(query):
    prompt = get_prompt_fully_qualified(query)
    return enricher.get_num_tokens(prompt)

fully_classified("dinosaur"), fully_classified("sofa loveseat"), fully_classified("sofa loveseat").classifications

(QueryClassification(keywords='dinosaur', hallucinated_classification=['Decor & Wall Art / Dinosaur Themed Decor / Dinosaur Wall Hangings and Art', 'Kids & Baby Furniture / Dinosaur Themed Bedroom Furniture / Dinosaur Bed Frames and Dressers', 'Toys & Games / Dinosaur Toys / Dinosaur Action Figures and Plush Toys', 'Outdoor & Garden / Dinosaur Garden Statues / Dinosaur Garden Decor', 'Party Supplies / Dinosaur Party Decorations / Dinosaur Balloons and Banners', 'Educational & Learning / Dinosaur Educational Toys / Dinosaur Puzzle and Learning Kits']),
 QueryClassification(keywords='sofa loveseat', hallucinated_classification=['Furniture / Living Room Furniture / Sofas & Loveseats / Loveseats and Small Sofas', 'Furniture / Modular & Sectional Sofas / Sectional Sofas', 'Furniture / Convertible & Sleeper Sofas / Sleeper Sofas and Futons', 'Furniture / Customizable & Modular Furniture / Modular Sofas and Sectionals', 'Furniture / Vintage & Retro Furniture / Retro Sofas and Loveseats', 'Fur

### Much lower number of tokens

We are using a cheaper model, and also we observe many fewer tokens

In [ ]:
get_num_tokens("sofa loveseat")

(380, 92)

### Run an example

Notice how the hallucinated classifications look very specific, but then look at how they are resolved.

**Consider** What problems exist here? Or might creep up?

In [ ]:
fully_classified("barnyard style doors").hallucinated_classification

['Home Improvement / Doors & Windows / Barn Doors & Sliding Doors / Barnyard Style Doors',
 'Furniture / Entryway Furniture / Barnyard Style Doors',
 'Decor / Rustic Decor / Barnyard Theme Accessories',
 'Architectural Elements / Barnyard Style Features / Barnyard Style Doors']

In [ ]:
fully_classified("barnyard style doors").classifications

[np.str_('Home Improvement / Doors & Door Hardware / Interior Doors / Sliding Interior Doors'),
 np.str_('Home Improvement / Doors & Door Hardware / Interior Doors'),
 np.str_('Décor & Pillows / Home Accessories / Decorative Objects'),
 np.str_('Home Improvement / Doors & Door Hardware / Door Hardware & Accessories / Barn Door Hardware')]

In [ ]:
fully_classified("barnyard style doors").cat_subcat

{'Décor & Pillows / Home Accessories',
 'Home Improvement / Doors & Door Hardware'}

### Redefine ground truth *for list use cases*

We defined a ground truth to look at jaccard + recall for specific categories

In [ ]:
from cheat_at_search.wands_data import labeled_query_products, queries

def get_top_categories(column, no_fit_label, cutoff=0.8):
    # Get relevant products per query
    top_products = labeled_query_products[labeled_query_products['grade'] == 2]

    # Aggregate top categories
    categories_per_query_ideal = top_products.groupby('query')[column].value_counts().reset_index()

    # Get as percentage of all categories for this query
    top_cat_proportion = categories_per_query_ideal.groupby(['query', column]).sum() / categories_per_query_ideal.groupby('query').sum()
    top_cat_proportion = top_cat_proportion.drop(columns=column).reset_index()

    # Only look at cases where the category is > 0.8
    top_cat_proportion = top_cat_proportion[top_cat_proportion['count'] > cutoff]
    top_cat_proportion[column].fillna(no_fit_label, inplace=True)
    ground_truth_cat = top_cat_proportion
    # Give No Category Fits to all others without dominant category
    ground_truth_cat = ground_truth_cat.merge(queries, how='right', on='query')[['query', column, 'count']]
    ground_truth_cat[column].fillna(no_fit_label, inplace=True)
    return ground_truth_cat

# The original ground truth by category
ground_truth_cat = get_top_categories('category', 'No Category Fits', cutoff=0.8)


ground_truth_cat_list = get_top_categories('category', 'No Category Fits', cutoff=0.05)

# Group by query, collect category into list
ground_truth_cat_list = ground_truth_cat_list.groupby('query').agg({'category': list}).reset_index()
# Remove empty items on "No Category Fits" from lists
ground_truth_cat_list['category'] = ground_truth_cat_list['category'].apply(lambda x: [y.strip() for y in x if y and y != 'No Category Fits'])
ground_truth_cat_list

/tmp/ipython-input-2868813019.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  top_cat_proportion[column].fillna(no_fit_label, inplace=True)
/tmp/ipython-input-2868813019.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

,query,category
0,12v dirt bike,[]
1,3 1/2 inch drawer pull,[Home Improvement]
2,3 piece rug set with runners,[Rugs]
3,3/4 size mattress,[]
4,30 inch bathroom vanity,[Home Improvement]
...,...,...
475,wrought tv stand,[Furniture]
476,zachary 72.5,[Furniture]
477,zakariyah twin platform,[Baby & Kids]
478,zen,"[Décor & Pillows, Home Improvement, Outdoor]"


In [ ]:
def get_preds(cat, column):
    if column == 'category':
        return cat.categories
    elif column == 'sub_category':
        return cat.sub_categories
    else:
        raise ValueError(f"Unknown column {column}")


def jaccard_sim(ground_truth, column, no_fit_label, classifier_fn):

    jaccard_sum = 0
    num_preds = 0
    num_w_preds = 0
    recall_sum = 0
    for _, row in ground_truth.sample(frac=1).iterrows():
        query = row['query']
        expected_category = set(row[column])
        cat = classifier_fn(query)
        preds = set(get_preds(cat, column))
        if (len(preds) == 0) or (preds == [no_fit_label]):
            print(f"Skipping {query}")
            continue

        if len(expected_category) > 0:
            recall = len(preds.intersection(expected_category)) / len(expected_category)
            print(f"Recall: {recall}")
            recall_sum += recall
            num_w_preds += 1
            print(f"Avg Recall (when pred): {recall_sum / num_w_preds}")


        jaccard = len(preds.intersection(expected_category)) / len(preds.union(expected_category))
        print(f"{query} -- pred:{preds} == expected:{expected_category}")
        print(f"Hallucinated -- {cat.hallucinated_classification}")
        print(f"Resolved -- {cat.classifications}")
        print(f"Jaccard: {jaccard}")
        jaccard_sum += jaccard
        num_preds += 1
        print(f"Avg Jaccard: {jaccard_sum / num_preds}")
        print(num_preds)
        print()
    return jaccard_sum / num_preds, num_preds / len(ground_truth)


jaccard, coverage = jaccard_sim(ground_truth_cat_list, 'category', 'No Category Fits', fully_classified)
jaccard, coverage

Recall: 1.0
Avg Recall (when pred): 1.0
small wardrobe grey -- pred:{'Storage & Organization', 'Baby & Kids', 'Furniture'} == expected:{'Furniture'}
Hallucinated -- ['Furniture / Bedroom Furniture / Wardrobes & Armoires / Small Wardrobes', 'Furniture / Storage & Organization / Closet Storage / Small Wardrobes', 'Home Decor / Color Specific Furniture / Grey Wardrobes', 'Furniture / Kids & Teen Bedroom Furniture / Small Wardrobes for Kids', 'Furniture / Entryway Furniture / Small Wardrobes for Entryways']
Resolved -- [np.str_('Furniture / Bedroom Furniture / Armoires & Wardrobes'), np.str_('Storage & Organization / Closet Storage & Organization / Closet Accessories'), np.str_('Furniture / Bedroom Furniture / Dressers & Chests / Beige Dressers & Chests'), np.str_('Baby & Kids / Toddler & Kids Bedroom Furniture / Baby & Kids Dressers'), np.str_('Furniture / Bedroom Furniture / Jewelry Armoires')]
Jaccard: 0.3333333333333333
Avg Jaccard: 0.3333333333333333
1

Recall: 1.0
Avg Recall (when pr

(0.4135292658730162, 1.0)

## Run Category search strategy with classifier

Identical to previous runs with a classifier of query -> list of classifications. We ask for the `categories` (which behind the scenes does teh embedding lookup). We then go through our usual boosting dance:

1. BM25 search
2. Boost category / subcategory matches

**Something to try (easy)** - change away from the MiniLM model and see if anything changes

**Something to try (advanced)** - What if we embedded the classification directly in the index, and searched that with the hallucinated classification?

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np


class CategorySearch(SearchStrategy):
    def __init__(self, products, query_to_cat,
                 name_boost=9.3,
                 description_boost=4.1,
                 category_boost=10,
                 sub_category_boost=5,
                 cat_subcat_boost=10):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        products['category'] = cat_split.apply(
            lambda x: x[0].strip() if len(x) > 0 else ""
        )
        products['subcategory'] = cat_split.apply(
            lambda x: x[1].strip() if len(x) > 1 else ""
        )
        self.index['category_snowball'] = SearchArray.index(
            products['category'], snowball_tokenizer
        )
        self.index['subcategory_snowball'] = SearchArray.index(
            products['subcategory'], snowball_tokenizer
        )

        self.index['cat_subcat'] = products['category'] + products['subcategory']
        self.index['cat_subcat'] = self.index['cat_subcat'].fillna('')
        self.index['cat_subcat_snowball'] = SearchArray.index(
            self.index['cat_subcat'], snowball_tokenizer
        )

        self.query_to_cat = query_to_cat
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.category_boost = category_boost
        self.sub_category_boost = sub_category_boost
        self.cat_subcat_boost = cat_subcat_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        structured = self.query_to_cat(query)
        tokenized = snowball_tokenizer(query)

        print(structured)

        # ****
        # Baseline BM25 search from before
        for token in tokenized:
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ****
        # If there's a subcategory, boost that by a constant amount
        for sub_category in structured.sub_categories:
            tokenized_subcategory = snowball_tokenizer(sub_category)
            subcategory_match = np.zeros(len(self.index))
            if tokenized_subcategory:
                subcategory_match = self.index['subcategory_snowball'].array.score(tokenized_subcategory) > 0
            bm25_scores[subcategory_match] += self.sub_category_boost

        # ****
        # If there's a category, boost that by a constant amount
        for category in structured.categories:
            print(category)
            tokenized_category = snowball_tokenizer(category)
            category_match = np.zeros(len(self.index))
            if tokenized_category:
                category_match = self.index['category_snowball'].array.score(tokenized_category) > 0
            bm25_scores[category_match] += self.category_boost

        # ***
        # If they occur togethere, boost by a constant amount
        for cat_subcat in structured.cat_subcat:
            tokenized_cat_subcat = snowball_tokenizer(cat_subcat)
            cat_subcat_match = np.zeros(len(self.index))
            if tokenized_cat_subcat:
                cat_subcat_match = self.index['cat_subcat_snowball'].array.score(tokenized_cat_subcat) > 0
            bm25_scores[cat_subcat_match] += self.cat_subcat_boost

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores

categorized_search = CategorySearch(products, fully_classified)
categorized_search.search('medium clips')

2025-10-15 13:41:38,602 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-10-15 13:41:38,617 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-10-15 13:41:38,620 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-10-15 13:41:53,006 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


keywords='medium clips' hallucinated_classification=['Hardware / Fasteners & Clips / Medium Clips', 'Home Organization / Cable & Wire Management / Medium Clips', 'Craft Supplies / Clamps & Clips / Medium Clips', 'Office & Desk Accessories / Paper Clips / Medium Clips', 'Industrial Supplies / Clamping Devices / Medium Clips']
Storage & Organization
Furniture
Home Improvement
Clips


(array([36813, 37397, 11295, 34050, 36809, 37396, 27888, 37398, 27886,
        36810]),
 array([85.35794735, 84.99754143, 72.97011852, 68.97158527, 68.62535572,
        67.39479828, 66.84630013, 66.0164814 , 65.3701973 , 64.83574677]))

In [ ]:
# categorized_search.sub_category_boost = 0
# categorized_search.category_boost = 0
# categorized_search.cat_subcat_boost = 0
top_k, scores = categorized_search.search('medium clips')
matched_products = products.iloc[top_k].copy()
matched_products['score'] = scores


matched_products[['product_name', 'category hierarchy', 'score']]

keywords='medium clips' hallucinated_classification=['Hardware / Fasteners & Clips / Medium Clips', 'Home Organization / Cable & Wire Management / Medium Clips', 'Craft Supplies / Clamps & Clips / Medium Clips', 'Office & Desk Accessories / Paper Clips / Medium Clips', 'Industrial Supplies / Clamping Devices / Medium Clips']
Storage & Organization
Furniture
Home Improvement
Clips


,product_name,category hierarchy,score
36813,"gem paper clips , plastic , medium size , 500/box",Clips/Clamps,85.357947
37397,"medium binder clips , 24/pack",Clips/Clamps,84.997541
11295,jaliah dining table with clipped corner medium...,Furniture / Kitchen & Dining Furniture / Dinin...,72.970119
34050,monett 10 piece medium clip set,NaN,68.971585
36809,"medium binder clips , steel wire , 5/8 '' capa...",Clips/Clamps,68.625356
37396,"paper clips , coated , jumbo , 80 clips per pa...",Clips/Clamps,67.394798
27888,fitting clips,Storage & Organization / Closet Storage & Orga...,66.846300
37398,"binder clips , medium,1-1/4 '' wide,5/8 '' cap...",Clips/Clamps,66.016481
27886,fretwell plastic hangers with clips for skirt/...,Storage & Organization / Closet Storage & Orga...,65.370197
36810,"regal clips , 100/box",Clips/Clamps,64.835747


In [ ]:
graded_fully_classified = run_strategy(categorized_search)
graded_fully_classified

keywords='salon chair' hallucinated_classification=['Furniture / Salon & Barber Chairs / Salon Chairs', 'Furniture / Styling & Barber Stations / Salon Stations', 'Furniture / Waiting Area Seating / Lounge Chairs for Salons', 'Furniture / Salon Equipment / Hair Styling Chairs', 'Furniture / Spa & Wellness Furniture / Massage & Treatment Chairs']


Searching:   0%|          | 1/480 [00:00<02:27,  3.26it/s]

Outdoor
Furniture
keywords='smart coffee table' hallucinated_classification=['Furniture / Smart Home Furniture / Connected Coffee Tables', 'Furniture / Living Room Furniture / Modern Coffee Tables with Tech Features', 'Home Automation / Smart Furniture / Interactive Coffee Tables', 'Furniture / Tech-Integrated Living Room Furniture / Smart Coffee Tables', 'Smart Home Devices / Connected Home Furniture / Coffee Tables with Built-in Technology']


Searching:   0%|          | 2/480 [00:00<02:38,  3.01it/s]

Furniture
keywords='dinosaur' hallucinated_classification=['Decor & Wall Art / Dinosaur Themed Decor / Dinosaur Wall Hangings and Art', 'Kids & Baby Furniture / Dinosaur Themed Bedroom Furniture / Dinosaur Bed Frames and Dressers', 'Toys & Games / Dinosaur Toys / Dinosaur Action Figures and Plush Toys', 'Outdoor & Garden / Dinosaur Garden Statues / Dinosaur Garden Decor', 'Party Supplies / Dinosaur Party Decorations / Dinosaur Balloons and Banners', 'Educational & Learning / Dinosaur Educational Toys / Dinosaur Puzzle and Learning Kits']


...

Searching: 100%|█████████▉| 479/480 [03:03<00:00,  2.62it/s]

Bed Accessories
Outdoor
Furniture
keywords='rack glass' hallucinated_classification=['Home Decor / Display & Storage / Glass Racks and Holders', 'Kitchen & Dining / Barware & Glassware Storage / Glass Racks', 'Furniture / Specialty Storage / Glassware Racks', 'Living Room Furniture / Entertainment & Media Storage / Glass Display Cabinets', 'Bathroom Accessories / Shower & Bath Storage / Glass Shelving Units']


Searching: 100%|██████████| 480/480 [03:03<00:00,  2.61it/s]

Home Improvement
Bed & Bath
Kitchen & Tabletop
Furniture


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,query_id,rank,query_class,id,label,grade,discounted_gain,idcg,dcg,ndcg
0,7465,hair salon chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,offers a wide selection of professional salon ...,fauxleathertype : pu|legheight-toptobottom:18|...,69.0,4.5,53.0,"[fauxleathertype : pu, legheight-toptobottom:1...",...,0,1,Massage Chairs,80.0,Exact,2.0,3.00,8.786905,8.10119,0.921962
1,25431,barberpub salon massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,supplierintendedandapproveduse : non residenti...,4.0,5.0,4.0,[supplierintendedandapproveduse : non resident...,...,0,2,Massage Chairs,29.0,Exact,2.0,1.50,8.786905,8.10119,0.921962
2,7468,mercer41 hair salon chair hydraulic styling ch...,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,mercer41 beauty offers a wide selection profes...,seatfillmaterial : foam|waterrepellant : no re...,1.0,5.0,1.0,"[seatfillmaterial : foam, waterrepellant : no ...",...,0,3,Massage Chairs,104.0,Exact,2.0,1.00,8.786905,8.10119,0.921962
3,39461,professional salon reclining massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,new and in a good condition . first-rate metal...,overalldepth-fronttoback:39.4|warrantylength:1...,NaN,NaN,NaN,"[overalldepth-fronttoback:39.4, warrantylength...",...,0,4,Massage Chairs,114.0,Exact,2.0,0.75,8.786905,8.10119,0.921962
4,9234,beauty salon task chair,NaN,Furniture / Office Furniture / Office Chairs,"applicable scene : office , home life , beauty...",overallheight-toptobottom:37|backcolor : brown...,NaN,NaN,NaN,"[overallheight-toptobottom:37, backcolor : bro...",...,0,5,Massage Chairs,32.0,Partial,1.0,0.20,8.786905,8.10119,0.921962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,40247,winn hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,are you looking for a safe and decorative solu...,overallheight-toptobottom:1.5|overallwidth-sid...,305.0,5.0,187.0,"[overallheight-toptobottom:1.5, overallwidth-s...",...,487,6,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4796,40245,podgorni hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,display and protect your delicate wine or marg...,overallheight-toptobottom:1.5|stemwarecapacity...,6.0,4.0,6.0,"[overallheight-toptobottom:1.5, stemwarecapaci...",...,487,7,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4797,7151,bataan hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,hang your stemware in style with this charming...,stemwarecapacity:18|material : wood|design : h...,103.0,4.5,78.0,"[stemwarecapacity:18, material : wood, design ...",...,487,8,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4798,40251,wellman chiraz hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,add flair to your wine decor with this set of ...,style : modern & contemporary|overallwidth-sid...,175.0,5.0,107.0,"[style : modern & contemporary, overallwidth-s...",...,487,9,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000


### Check results - Observe still healthy NDCG bump

NDCG still pretty good (not as good as sending the Literal list). But still pretty healthy increase.

In [ ]:
ndcgs(graded_bm25).mean(), ndcgs(graded_fully_classified).mean()

(np.float64(0.5411098691836396), np.float64(0.5586509656926943))

In [ ]:
deltas = ndcg_delta(graded_fully_classified, graded_bm25)

In [ ]:
sig_improved = len(deltas[deltas > 0.1])
print(f"Num Significatly Improved: {sig_improved}")
deltas[deltas > 0.1]

Num Significatly Improved: 34


,ndcg
query,
drum picture,0.506435
non slip shower floor tile,0.458113
desk for kids,0.412546
turquoise chair,0.403107
modern outdoor furniture,0.365533
twin bed frame,0.359391
wood rack wide,0.346159
liberty hardware francisco,0.292914
storage dresser,0.270289


In [ ]:
sig_harmed = len(deltas[deltas < -0.1])
print(f"Num Significatly Harmed: {sig_harmed}")
print(f"Prop improved/harmed: {sig_improved / (sig_harmed + sig_improved)} | {sig_harmed / (sig_harmed + sig_improved)}")
deltas[deltas < -0.1]

Num Significatly Harmed: 6
Prop improved/harmed: 0.85 | 0.15


,ndcg
query,
bedroom accessories,-0.113173
zodiac pillow,-0.113806
papasan chair frame only,-0.123290
bathroom freestanding cabinet,-0.155670
outdoor lounge cushions,-0.223547
sugar canister,-0.224721


### Look at a query

In [ ]:
QUERY = "outdoor lounge cushions"
fully_classified(QUERY), fully_classified(QUERY).cat_subcat

(QueryClassification(keywords='outdoor lounge cushions', hallucinated_classification=['Outdoor / Outdoor Comfort & Decor / Lounge Cushions & Pillows / Outdoor Lounge Cushions', 'Furniture / Patio & Garden Furniture / Lounge Seating / Outdoor Lounge Cushions', 'Home Decor / Textiles / Cushions & Pillows / Outdoor Lounge Cushions', 'Garden & Patio / Outdoor Furniture Accessories / Cushions & Pillows / Lounge Cushions']),
 {'Outdoor / Outdoor & Patio Furniture', 'Outdoor / Outdoor Décor'})

In [ ]:
ground_truth_cat[ground_truth_cat['query'] == QUERY]

,query,category,count
179,outdoor lounge cushions,,0.805556


In [ ]:
graded_fully_classified[graded_fully_classified['query'] == QUERY][['product_name', 'category hierarchy', 'grade', 'score']]

,product_name,category hierarchy,grade,score
1790,classic outdoor lounge chair cushion,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,89.672464
1791,outdoor reclining chaise lounge with cushion,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,87.930735
1792,iola outdoor reclining chaise lounge with cushion,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,87.345481
1793,outdoor lounge chair cushion,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,86.526578
1794,make it rain cerulean indoor/outdoor chaise lo...,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,85.435493
1795,annoeska outdoor reclining chaise lounge with ...,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,85.104139
1796,french edge indoor/outdoor lounge chair cushion,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,83.353908
1797,achary outdoor reclining chaise lounge with cu...,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,82.629874
1798,alexandra outdoor reclining chaise lounge with...,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,82.133712
1799,cara indooroutdoor chaise lounge cushion,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,81.857544


In [ ]:
graded_bm25[graded_bm25['query'] == QUERY][['product_name', 'category hierarchy', 'grade', 'score']]

,product_name,category hierarchy,grade,score
1790,outdoor chaise lounge cushion,NaN,2.0,79.604049
1791,dorrington outdoor chaise lounge cushion,NaN,2.0,75.647600
1792,replacement pillows outdoor lounge chair cushion,NaN,2.0,75.266675
1793,indoor/outdoor sunbrella chaise lounge cushion,NaN,2.0,74.830838
1794,classic outdoor lounge chair cushion,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0,74.672464
1795,tropical outdoor lounge chair cushion,NaN,2.0,73.328272
1796,outdoor reclining chaise lounge with cushion,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,72.930735
1797,jermaine outdoor lounge chair cushion,NaN,2.0,72.379733
1798,iola outdoor reclining chaise lounge with cushion,Outdoor / Outdoor & Patio Furniture / Outdoor ...,1.0,72.345481
1799,jermaine outdoor chaise lounge cushion,NaN,2.0,72.293746


In [ ]:
graded_bm25.iloc[3103]

,3103
product_id,11295
product_name,jaliah dining table with clipped corner medium...
product_class,Dining Tables
category hierarchy,Furniture / Kitchen & Dining Furniture / Dinin...
product_description,a sleek silhouette makes this dining table a p...
product_features,estimatedtimetosetup:10|tablemechanism : fixed...
rating_count,NaN
average_rating,NaN
review_count,NaN
features,"[estimatedtimetosetup:10, tablemechanism : fix..."


In [ ]:
products[products['product_id'] == 11295]['category hierarchy'].iloc[0]

'Furniture / Kitchen & Dining Furniture / Dining Tables & Seating / Kitchen & Dining Tables'

In [ ]:
QueryClassification.model_json_schema()

{'description': 'Represents a classification of a product.\n\nIn this case, hallucinated, something the model is making up that looks like one\nof our classifications',
 'properties': {'keywords': {'description': 'The original search query keywords sent in as input',
   'title': 'Keywords',
   'type': 'string'},
  'hallucinated_classification': {'description': 'The classification you created for the query.',
   'items': {'type': 'string'},
   'title': 'Hallucinated Classification',
   'type': 'array'}},
 'required': ['keywords', 'hallucinated_classification'],
 'title': 'QueryClassification',
 'type': 'object'}

In [ ]:
top_k, scores = categorized_search.search('medium clips')
matched_products = products.iloc[top_k].copy()
matched_products['score'] = scores

matched_products[['product_name', 'category hierarchy', 'score']]

keywords='medium clips' hallucinated_classification=['Hardware / Fasteners & Clips / Medium Clips', 'Home Organization / Cable & Wire Management / Medium Clips', 'Craft Supplies / Clamps & Clips / Medium Clips', 'Office & Desk Accessories / Paper Clips / Medium Clips', 'Industrial Supplies / Clamping Devices / Medium Clips']
Storage & Organization
Furniture
Home Improvement
Clips


,product_name,category hierarchy,score
36813,"gem paper clips , plastic , medium size , 500/box",Clips/Clamps,85.357947
37397,"medium binder clips , 24/pack",Clips/Clamps,84.997541
11295,jaliah dining table with clipped corner medium...,Furniture / Kitchen & Dining Furniture / Dinin...,72.970119
34050,monett 10 piece medium clip set,NaN,68.971585
36809,"medium binder clips , steel wire , 5/8 '' capa...",Clips/Clamps,68.625356
37396,"paper clips , coated , jumbo , 80 clips per pa...",Clips/Clamps,67.394798
27888,fitting clips,Storage & Organization / Closet Storage & Orga...,66.846300
37398,"binder clips , medium,1-1/4 '' wide,5/8 '' cap...",Clips/Clamps,66.016481
27886,fretwell plastic hangers with clips for skirt/...,Storage & Organization / Closet Storage & Orga...,65.370197
36810,"regal clips , 100/box",Clips/Clamps,64.835747


In [ ]:
products[products['category hierarchy'] == 'Clips/Clamps']